# A 10-minute Buy-and-Hold Template

This simple template provides basic instructions for helping you within 10 minutes to develop and submit a buy-and-hold strategy on liquid assets.

In [ ]:
# Import basic libraries for manipulating data.

# Please refer to xarray.pydata.org for xarray documentation.

# xarray works optimally with N-dimensional datasets in Python
# and is well suited for financial datasets with labels "time",
# "field" and "asset". xarray data structures can also be easily
# converted to pandas dataframes.

import xarray as xr

import numpy as np
import pandas as pd

# Import quantnet libraries.

import qnt.data as qndata          # data loading and manipulation
import qnt.stepper as qnstepper    # strategy definition
import qnt.stats as qnstats        # key statistics
import qnt.graph as qngraph        # graphical tools
import qnt.forward_looking as qnfl # forward looking checking

# display function for fancy displaying:
from IPython.display import display

In [ ]:
# Load all available asset names since given date.

assets = qndata.load_assets(min_date="2010-01-01")

assets_names = [i["id"] for i in assets]

# Load all available data since given date.

# It is possible to set a max_date in the call in order to
# develop the system on a limited in-sample period and later
# test the system on unseen data after max_date.

# A submission will be accepted only if no max_date is set,
# as submissions will be evaluated on live data on a daily basis.

data = qndata.load_data(min_date="2010-01-01",
                        max_date="2019-08-13", # You should not limit max date for final calculation!
                        dims=("time", "field", "asset"),
                        assets=assets_names,
                        forward_order=True)

In [ ]:
# A buy-and-hold strategy on liquid assets allocates
# constant fractions of capital to all liquid assets.
# Here xarray data structures are converted to pandas
# dataframes for simplicity in order to describe the
# development process.

# xarray.DataArray are converted to pandas dataframes:
is_liquid = data.loc[:,"is_liquid",:].to_pandas()

# set and normalize weights:
weights = is_liquid.div(is_liquid.abs().sum(axis=1, skipna=True), axis=0)
weights = weights.fillna(0.0)

# set max columns and rows for display function
with pd.option_context("display.max_rows", 5, "display.max_columns", 10): 
    display(weights)

In [ ]:
# check that we are fully invested <=> sum of abs(weights) = 1:
with pd.option_context("display.max_rows", 5): 
    display(weights.abs().sum(axis=1))

In [ ]:
# visualize positions for a selected asset:
aapl_frac = weights["NASDAQ:AAPL"]
qngraph.make_plot(aapl_frac.index, aapl_frac, name= "aapl")

# Statistics

In [ ]:
#convert to xarray before statistics calculation

output = weights.unstack().to_xarray()
output 

In [ ]:
# Calculate statistics on a rolling basis.

# Transactions are punished with slippage equal to a given
# fraction of ATR14 (read more about slippage in our full
# Strategy Buy and Hold template). We evaluate submissions
# using 5% of ATR14 for slippage.

# Mean return, volatility and Sharpe ratio are computed on a
# rolling basis using a lookback period of 3 years.

stat = qnstats.calc_stat(data, output, slippage_factor=0.05)

display(stat.to_pandas().tail())

In [ ]:
def print_stat(stat):
    """Prints selected statistical key indicators:
       - the global Sharpe ratio of the strategy;
       - the global mean profit;
       - the global volatility;
       - the maximum drawdown.

       Note that Sharpe ratio, mean profit and volatility
       apply to  max simulation period, and not to the
       rolling basis of 3 years.
    """

    days = len(stat.coords["time"])
    
    returns = stat.loc[:, "relative_return"]

    equity = stat.loc[:, "equity"]
    
    sharpe_ratio = qnstats.calc_sharpe_ratio_annualized(
        returns,
        max_periods=days,
        min_periods=days).to_pandas().values[-1]

    profit = (qnstats.calc_mean_return_annualized(
        returns,
        max_periods=days,
        min_periods=days).to_pandas().values[-1])*100.0

    volatility = (qnstats.calc_volatility_annualized(
        returns,
        max_periods=days,
        min_periods=days).to_pandas().values[-1])*100.0

    max_ddown = (qnstats.calc_max_drawdown(
        qnstats.calc_underwater(equity)).to_pandas().values[-1])*100.0

    print("Sharpe Ratio         : ", "{0:.3f}".format(sharpe_ratio))
    print("Mean Return [%]      : ", "{0:.3f}".format(profit))
    print("Volatility [%]       : ", "{0:.3f}".format(volatility))
    print("Maximum Drawdown [%] : ", "{0:.3f}".format(-max_ddown))

print_stat(stat)

In [ ]:
# show plot with profit and losses:
performance = stat.to_pandas()["equity"]
qngraph.make_plot_filled(performance.index, performance, name="PnL (Equity)", type="log")

In [ ]:
# show underwater chart:
UWchart = stat.to_pandas()["underwater"]
qngraph.make_plot_filled(UWchart.index, UWchart, color="darkred", name="Underwater Chart", range_max=0)

In [ ]:
# show rolling Sharpe ratio on a 3-year basis:
SRchart = stat.to_pandas()["sharpe_ratio"].iloc[(252*3):]
qngraph.make_plot_filled(SRchart.index, SRchart, color="#F442C5", name="Rolling SR")

In [ ]:
# show bias chart:
biaschart = stat.to_pandas()["bias"]
qngraph.make_plot_filled(biaschart.index, biaschart, color="#5A6351", name="Bias Chart")

# Improvement

In [ ]:
# Well, sharpe ratio of this strategy is low...

stat.sel(field='sharpe_ratio').to_pandas().tail()

In [ ]:
# Let's try to improve it using statistics per asset.
# At first, calculate stats per asset:

stat_per_asset = qnstats.calc_stat(data, output, slippage_factor=0.05, per_asset=True)
stat_per_asset.dims

In [ ]:
# For example, AAPL have good sharpe ratio:
stat_per_asset.sel(asset='NASDAQ:AAPL', field='sharpe_ratio').to_pandas().tail()

In [ ]:
# Let's build the output only with "good" "short term" and "long term" Sharpe ratios.
# Sharpe ratio is "good" when its average more then 0.
#
# This is only example of a heuristic which can improve you strategy 
# using statistics per asset. 
# I believe that you can invent a new better way to do it =)

short_term = 43
long_term = short_term*3

stat_per_asset_short_term = qnstats.calc_stat(data, output, max_periods=short_term, per_asset = True)
stat_per_asset_long_term = qnstats.calc_stat(data, output, max_periods=long_term, per_asset = True)

avg_short_term_sr = stat_per_asset_short_term.sel(field='sharpe_ratio')\
    .rolling(time=short_term, min_periods=short_term*19//20)\
    .mean() # min periods allows to pass small holes in data
avg_long_term_sr = stat_per_asset_long_term.sel(field='sharpe_ratio')\
    .rolling(time=long_term, min_periods=long_term*19//20)\
    .mean()

output2 = output
output2 = output2.where(avg_short_term_sr > 0)
output2 = output2.where(avg_long_term_sr > 0)
output2 = output2 / abs(output2).sum('asset')

In [ ]:
# print stats

stat2 = qnstats.calc_stat(data, output2)

print("Old stats:")
print("-\n3y SR:")
print(stat.sel(field='sharpe_ratio').to_pandas().tail())
print("-\nGlobal:")
print_stat(stat)

print("---")

print("New stats:")
print("-\n3y SR:")
print(stat2.sel(field='sharpe_ratio').to_pandas().tail())
print("-\nGlobal:")
print_stat(stat2)

Well, it only adds +0.18(\~23%) to the Sharpe ratio of 3 years, but adds +0.2(\~34%) to the Sharpe Ratio of the entire data period. Sometimes it can allow you to go through the Sharpe ratio filter or win a competition.

## Checks

In [ ]:
# Use the function from 'qnfl' ensures that no forward-looking
# is taking place. 
def strategy():
    """
    it is the same strtegy, but implemented with xarray
    Entire code of strategy calculation is collected here.
    """

    # data loading
    data = qndata.load_data(
        min_date="2010-01-01", 
        # max_date="2019-08-13", # You should not limit max_date for final calculations!
        dims=("time", "field", "asset"), 
        forward_order=True
    )
    
    # buy and hold strategy output calculation
    output1 = data.loc[::,"is_liquid",:]  
    output1 = output1 / abs(output1).sum('asset')
    
    # output improvement which use statistics per asset 
    short_term = 43
    long_term = short_term*3

    stat_per_asset_short_term = qnstats.calc_stat(data, output1, max_periods=short_term, per_asset = True)
    stat_per_asset_long_term = qnstats.calc_stat(data, output1, max_periods=long_term, per_asset = True)

    avg_short_term_sr = stat_per_asset_short_term.sel(field='sharpe_ratio')\
        .rolling(time=short_term, min_periods=short_term*19//20)\
        .mean() # min periods allows to pass small holes in data
    avg_long_term_sr = stat_per_asset_long_term.sel(field='sharpe_ratio')\
        .rolling(time=long_term, min_periods=long_term*19//20)\
        .mean()

    output2 = output
    output2 = output2.where(avg_short_term_sr > 0)
    output2 = output2.where(avg_long_term_sr > 0)
    output2 = output2 / abs(output2).sum('asset')
    
    return output2

# This function runs strategy twice on the different periods: 
# the entire data and data the with a cropped last half year.
# After that this function compares outputs. 
# Overlapped outputs must be same.
output_final = qnfl.load_data_calc_output_and_check_forward_looking(strategy)

In [ ]:
# correlation check
# your strategy should not correlate with other strategies before submission
qnstats.print_correlation(output_final, data)

# Write output

In [ ]:
# Finally, we write the last mandatory step for submission,
# namely writing output to file:

qndata.write_output(output_final)

At this stage code is ready for submission. Just click on the submission button in your account page and we will evaluate your strategy live on our servers!

For speeding up evaluation, you can consider submitting a copy with only relevant steps and excluding plots and checks. Your code in the final notebook you submit can be as simple as the following:

In [ ]:
import xarray as xr
import qnt.data as qndata

# data loading
data = qndata.load_data(
    min_date="2010-01-01", 
    # max_date="2019-08-13", # You should not limit max_date for final calculations
    dims=("time", "field", "asset"), 
    forward_order=True
)

# buy and hold strategy output calculation
output1 = data.loc[::,"is_liquid",:]  
output1 = output1 / abs(output1).sum('asset')

# output improvement which use statistics per asset 
short_term = 43
long_term = short_term*3

stat_per_asset_short_term = qnstats.calc_stat(data, output1, max_periods=short_term, per_asset = True)
stat_per_asset_long_term = qnstats.calc_stat(data, output1, max_periods=long_term, per_asset = True)

avg_short_term_sr = stat_per_asset_short_term.sel(field='sharpe_ratio')\
    .rolling(time=short_term, min_periods=short_term*19//20)\
    .mean() # min periods allows to pass small holes in data
avg_long_term_sr = stat_per_asset_long_term.sel(field='sharpe_ratio')\
    .rolling(time=long_term, min_periods=long_term*19//20)\
    .mean()

output2 = output1
output2 = output2.where(avg_short_term_sr > 0)
output2 = output2.where(avg_long_term_sr > 0)
output2 = output2 / abs(output2).sum('asset')

qndata.write_output(output2)